# Krzysztof Nalepa
## Sprawozdanie z ćwiczenia 9
### Rozwiązywanie układów równań liniowych

#### Zadanie 1 
Zaimplemenuj metodę eliminacji Gaussa bez pivotingu i z pivotingiem dla układu równań o dowolnym rozmiarze. 

In [0]:
def gauss_elimination_method_without_pivoting(A, B):
    if len(A) != len(A[0]):
        return None
    n = len(A)

    for row in range(0, n - 1):
        for i in range(row + 1, n):
            f = A[i][row] / A[row][row]
            for j in range(row, n):
                A[i][j] -= f * A[row][j]
            B[i] -= f * B[row]

    result = [0] * n
    result[n - 1] = B[n - 1] / A[n - 1][n - 1]
    for row in range(n - 2, -1, -1):
        sum = B[row]
        for j in range(row + 1, n):
            sum -= A[row][j] * result[j]
        result[row] = sum / A[row][row]

    return result

def gauss_elimination_method_with_pivoting(A, B):
    if len(A) != len(A[0]):
        return None
    n = len(A)
    M = A
    i = 0
    for row in M:
        row.append(B[i])
        i += 1

    for k in range(n):
        for i in range(k, n):
            if abs(M[i][k]) > abs(M[k][k]):
                M[k], M[i] = M[i], M[k]

        for j in range(k + 1, n):
            q = M[j][k] / M[k][k]
            for m in range(k, n + 1):
                M[j][m] -= q * M[k][m]

    result = [0] * n

    result[n - 1] = M[n - 1][n] / M[n - 1][n - 1]
    for i in range(n - 1, -1, -1):
        z = 0
        for j in range(i + 1, n):
            z += M[i][j] * result[j]
        result[i] = (M[i][n] - z) / M[i][i]
    return result


#### Zadanie 2
Zademonstruj działanie algorytmu na macierzy o rozmiarze 5 x 5. Zademonstruj w jakiej sytuacji potrzebny jest pivoting i jak działa. 

In [5]:
import copy

A = [[2, -7, 0, 3, 2], 
     [-3, 0, 6, 1, 6], 
     [5, -1, 5, 1, 2],
     [1, 6, 2, -1, 2],
     [4, 2, 1, -5, 1]]
B = [7, 3, 6, 1, 2]

x = gauss_elimination_method_without_pivoting(copy.deepcopy(A), copy.deepcopy(B))
print(x)

y = gauss_elimination_method_with_pivoting(copy.deepcopy(A), copy.deepcopy(B))
print(y)


[0.9142228739002936, -0.2104105571847507, -0.32624633431085015, 0.42448680351906154, 1.2126099706744866]
[0.9142228739002931, -0.21041055718475074, -0.32624633431085026, 0.42448680351906165, 1.2126099706744866]


In [7]:
import numpy as np
from tabulate import tabulate

A = [[0.00000000000000062, 3123, 335, 134, 3123], 
      [0.123, 0.00000000000000012, 1231, 9201, 1231], 
      [1123, 17123, 0.00000000000000013, 31230, 1231],
      [123, 1235, 12512, 0.00000000000000032, 2352],
      [1235, 123521, 12451, 12541, 0.00000000000000012]]
B = [7, 3, 6, 2, 1]
AN = np.array(A)
BN = np.array(B)

x = gauss_elimination_method_without_pivoting(copy.deepcopy(A), copy.deepcopy(B))

z = np.linalg.solve(AN, BN).tolist()

y = gauss_elimination_method_with_pivoting(copy.deepcopy(A), copy.deepcopy(B))
table = [['Without pivoting'] + x, ['With pivoting'] + y,  ['Scipy value'] + z]

print(tabulate(table, headers=['Type', 'a', 'b', 'c', 'd', 'e']), "\n")


Type                        a            b             c            d           e
----------------  -----------  -----------  ------------  -----------  ----------
Without pivoting  1.43255      0.000465717  -0.00289898   0.000393082  0.00206982
With pivoting     0.000838886  2.09293e-05  -0.000272879  6.19069e-05  0.00224712
Scipy value       0.000838886  2.09293e-05  -0.000272879  6.19069e-05  0.00224712 



Jak widać dla drugiego przykładu jest spora różnica między wartościami wyliczanymi bez pivotingu i z pivotingiem. Wynika ona z tego, że od małych liczb odejmujemy dość duże liczby co powoduje błędy wynikające z zaogrąglania.

#### Zadanie 3
Podaj teorytyczną złożoność obliczeniową algorytmu eliminacji Gaussa. Przeprowadź testy wydajności swojego algorytmu sprawdzając jego działanie dla różnych rozmiarów macierzy (testy powinny być wykonane poza środowiskiem jupyter). Aby wygenerować układ równań, wygeneruj wektor rozwiązań i macierz współczynników losując wartości (skorzystaj z funkcji poznanych w Ćwiczeniu 2) i następnie oblicz wektor wyrazów wolnych. 


Teoretyczna złożoność obliczeniowa algorytmu eliminacji Gaussa to O($n^3$). Do wykonania pomiarów i wygenerowania wykresów użyłem poniższego skryptu

In [0]:
import numpy as np
import time
from random import randrange
import copy
import matplotlib.pyplot as plt


def random_matrix(rows, columns):
    return np.random.rand(rows, columns)


def random_vector(size):
    return np.random.rand(size)


def time_check():
    n = randrange(100, 500)
    A = random_matrix(n, n).tolist()
    B = random_vector(n).tolist()

    start = time.time()
    gauss_elimination_method_without_pivoting(copy.deepcopy(A), copy.deepcopy(B))
    stop = time.time()
    without_pivot = stop - start

    start = time.time()
    gauss_elimination_method_with_pivoting(copy.deepcopy(A), copy.deepcopy(B))
    stop = time.time()
    with_pivot = stop - start
    return without_pivot, with_pivot, n


# Main program starts here
if __name__ == '__main__':
    size_list = []
    with_list = []
    without_list = []
    for i in range(100):
        without_pivot, with_pivot, n = time_check()
        without_list.append(with_pivot)
        with_list.append(with_pivot)
        size_list.append(n)

    f = open("dane.txt", "w+")
    for i in range(100):
        f.write(f"Matrix size {size_list[i]} \t With pivoting {with_list[i]} \t Without pivoting {without_list[i]}\n")
    f.close()

    plt.plot(size_list, without_list, 'bo', label="Without pivoting")
    plt.xlabel("Matrix size")
    plt.ylabel("Time")
    plt.title("Pomiary czasu metody eliminacji Gaussa")
    plt.legend()
    plt.savefig("wykres1")

    plt.close()
    plt.plot(size_list, with_list, 'ro', label="With pivoting")
    plt.xlabel("Matrix size")
    plt.ylabel("Time")
    plt.title("Pomiary czasu metody eliminacji Gaussa")
    plt.legend()
    plt.savefig("wykres2")


Wykresy oraz pomiary umieściłem w osobnym pliku pdf